In [ ]:
import tensorflow as tf

# Download and import the MIT Introduction to Deep Learning package
# !pip install mitdeeplearning --quiet
import mitdeeplearning as mdl

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
a = tf.constant(15)
b = tf.constant(61)
c1 = tf.add(a, b)
c2 = a + b
print(c1)
print(c2)

In [ ]:
def func(a, b):
    c = tf.add(a, b)
    d = tf.subtract(b, 1)
    e = tf.multiply(c, d)
    return e

a, b = 1.5, 2.5
e_out = func(a, b)
print(e_out)

In [ ]:
class OurDenseLayer(tf.keras.layers.Layer):
    def __init__(self, n_output_nodes):
        super(OurDenseLayer, self).__init__()
        self.n_output_nodes = n_output_nodes
        
    def build(self, input_shape):
        d = int(input_shape[-1])
        self.W = self.add_weight(name="weight", shape=[d, self.n_output_nodes])
        self.b = self.add_weight(name="bias", shape=[1, self.n_output_nodes])
        
    def call(self, x):
        z = tf.add(tf.matmul(x, self.W), self.b)
        y = tf.sigmoid(z)
        return y
    
tf.keras.utils.set_random_seed(1)
layer = OurDenseLayer(3)
layer.build((1,2))
x_input = tf.constant([[1,2.]], shape=(1,2))
y = layer.call(x_input)
print(y.numpy())
#mdl.lab1.test_custom_dense_layer_output(y)
        

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

n_output_nodes = 3
model = Sequential()
dense_layer = Dense(n_output_nodes, activation='sigmoid')
model.add(dense_layer)

x_input = tf.constant([[1,2.]], shape=(1,2))
model_output = model(x_input).numpy()
print(model_output)

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense

class SubclassModel(tf.keras.Model):
    def __init__(self, n_output_nodes):
        super(SubclassModel, self).__init__()
        self.dense_layer = Dense(n_output_nodes, activation='sigmoid')
        
    def call(self, inputs):
        return self.dense_layer(inputs)
    
n_output_nodes = 3
model = SubclassModel(n_output_nodes)
x_input = tf.constant([[1, 2.]], shape=(1,2))
print(model.call(x_input))

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense

class IdentityModel(tf.keras.Model):
    def __init__(self, n_output_nodes):
        super(IdentityModel, self).__init__()
        self.dense_layer = Dense(n_output_nodes, activation='sigmoid')
        
    def call(self, inputs, isidentity=False):
        return self.dense_layer(inputs) if not isidentity else inputs
    
n_output_nodes = 3
model = IdentityModel(n_output_nodes)
x_input = tf.constant([[1,2.]], shape=(1,2))
out_activate = model.call(x_input)
out_identity = model.call(x_input, True)

print("Network output with activation: {}; network identity output: {}".format(out_activate.numpy(), out_identity.numpy()))

In [ ]:
x = tf.Variable(3.0)
with tf.GradientTape() as tape:
    y = x * x
dy_dx = tape.gradient(y, x)
assert dy_dx.numpy() == 6.0

In [ ]:
### Function minimization with automatic differentiation and SGD ###

# Initialize a random value for our initial x
x = tf.Variable([tf.random.normal([1])])
print(f"Initializing x={x.numpy()}")

learning_rate = 1e-2 # learning rate for SGD
history = []
# Define the target value
x_f = 4

# We will run SGD for a number of iterations. At each iteration, we compute the loss,
#   compute the derivative of the loss with respect to x, and perform the SGD update.
for i in range(500):
  with tf.GradientTape() as tape:
    loss = (x - x_f)**2

  # loss minimization using gradient tape
  grad = tape.gradient(loss, x) # compute the derivative of the loss with respect to x
  new_x = x - learning_rate*grad # sgd update
  x.assign(new_x) # update the value of x
  history.append(x.numpy()[0])

# Plot the evolution of x as we optimize towards x_f!
plt.plot(history)
plt.plot([0, 500],[x_f,x_f])
plt.legend(('Predicted', 'True'))
plt.xlabel('Iteration')
plt.ylabel('x value')